参考资料：[sklearn: 利用TruncatedSVD做文本主题分析](https://blog.csdn.net/blmoistawinde/article/details/83446529)

In [1]:
# Until the Day 作者：林俊杰
docs = ["In the middle of the night",
        "When our hopes and fears collide",
        "In the midst of all goodbyes",
        "Where all human beings lie",
        "Against another lie"]

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(docs)

In [4]:
terms = vectorizer.get_feature_names()
# terms

In [5]:
len(terms)

20

In [9]:
X.toarray()

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.3319971 , 0.        , 0.41150185, 0.        , 0.41150185,
        0.3319971 , 0.        , 0.6639942 , 0.        , 0.        ],
       [0.        , 0.        , 0.40824829, 0.        , 0.        ,
        0.40824829, 0.40824829, 0.        , 0.40824829, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.40824829, 0.        , 0.40824829, 0.        ],
       [0.        , 0.37601977, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.46606681, 0.        , 0.        ,
        0.37601977, 0.        , 0.        , 0.46606681, 0.        ,
        0.37601977, 0.        , 0.37601977, 0.        , 0.        ],
       [0.        , 0.38898761, 0.        , 0.        , 0.48214012,
        0.        , 0.        , 0.        , 0.        , 0.48214012,
        0.        , 0.38898761, 0.        , 0

In [8]:
# 5 个数据，每个数据 20 个维度，每个维度是一个单词
X.toarray().shape

(5, 20)

In [11]:
# namely LSA/LSI 潜在语义分析
from sklearn.decomposition import TruncatedSVD

# 设定主题数为 3
n_pick_topics = 3
lsa = TruncatedSVD(n_pick_topics)

In [12]:
X2 = lsa.fit_transform(X)

In [14]:
import numpy as np

np.set_printoptions(suppress=True, precision=4)

In [15]:
X2

array([[ 0.8266, -0.2469,  0.    ],
       [ 0.    , -0.    ,  1.    ],
       [ 0.8667, -0.0909, -0.    ],
       [ 0.2801,  0.7287, -0.    ],
       [ 0.1031,  0.764 ,  0.    ]])

使用 `TruncatedSVD` 把原先规模为（文本数，词汇数）的特征矩阵 `X` 化为规模为（文本数，主题数）的新特征矩阵 `X2`；

由于主题数一般比词汇数少，这一方法也可以用来降维，用以后续进行分类或聚类操作。

这个时候，列数就成为 3 ，表示 3 个主题，这就是降维。

`X2[i,t]`为第 `i` 篇文档在第 `t` 个主题上的分布，所以该值越高的文档 `i`，可以认为在主题 `t` 上更有代表性，我们便以此筛选出最能代表该主题的文档。

https://blog.csdn.net/xiayangmian4130/article/details/86162972

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 文本格式要求,中文需要使用分词工具预处理
document = ['我们 走 在 热闹 的 大街 上', 
            '当 我们 走到 街 中心 时', 
            '哪里 真是 人山人海', 
            '热闹非凡']

In [19]:
tfidf_model = TfidfVectorizer(
    token_pattern=r"(?u)\b\w+\b", sublinear_tf=True,
    use_idf=True).fit(document)
# 得到的是一个类似于以三元组方式存储的稀疏矩阵,可以使用'.todense()'还原出原矩阵
sparse_result = tfidf_model.transform(document)

# 这一步可以得到一个词汇列表,可以使用索引还原出对应的词
print(tfidf_model.vocabulary_)

# 该方法可以得到所有的特征的名称列表
terms = tfidf_model.get_feature_names()

{'我们': 7, '走': 14, '在': 4, '热闹': 9, '的': 11, '大街': 5, '上': 0, '当': 6, '走到': 15, '街': 13, '中心': 1, '时': 8, '哪里': 3, '真是': 12, '人山人海': 2, '热闹非凡': 10}


In [23]:
tfidf_model.vocabulary_

{'上': 0,
 '中心': 1,
 '人山人海': 2,
 '哪里': 3,
 '在': 4,
 '大街': 5,
 '当': 6,
 '我们': 7,
 '时': 8,
 '热闹': 9,
 '热闹非凡': 10,
 '的': 11,
 '真是': 12,
 '街': 13,
 '走': 14,
 '走到': 15}

In [22]:
sparse_result.toarray().shape

(4, 16)

In [31]:
n_pick_topics = 3  # 设定主题数为3
svd = TruncatedSVD(n_pick_topics)
X = svd.fit_transform(sparse_result)
X

array([[ 0.7423,  0.    , -0.    ],
       [ 0.7423,  0.    ,  0.    ],
       [-0.    ,  0.4061,  0.9138],
       [-0.    ,  0.9138, -0.4061]])

In [32]:
n_pick_docs = 2
topic_docs_id = [
    X[:, t].argsort()[:-(n_pick_docs + 1):-1] for t in range(n_pick_topics)
]
topic_docs_id
# 3 个主题

[array([0, 1]), array([3, 2]), array([2, 1])]

In [27]:
n_pick_keywords = 4
topic_keywords_id = [svd.components_[t].argsort()[:-(n_pick_keywords+1):-1] for t in range(n_pick_topics)]
topic_keywords_id

# 3 个主题

[array([ 7,  1, 15, 13]), array([10,  2, 12,  3]), array([ 2, 12,  3, 14])]

In [41]:
reverse_vocabulary = {
    value: key
    for key, value in tfidf_model.vocabulary_.items()
}

reverse_vocabulary

{0: '上',
 1: '中心',
 2: '人山人海',
 3: '哪里',
 4: '在',
 5: '大街',
 6: '当',
 7: '我们',
 8: '时',
 9: '热闹',
 10: '热闹非凡',
 11: '的',
 12: '真是',
 13: '街',
 14: '走',
 15: '走到'}

In [44]:
for words in topic_keywords_id:
    print([reverse_vocabulary[i] for i in words])

['我们', '中心', '走到', '街']
['热闹非凡', '人山人海', '真是', '哪里']
['人山人海', '真是', '哪里', '走']
